In [1]:
# Importando as bibliotecas necessárias
import os
import sys

import pandas as pd

sys.path.append(os.path.abspath("../../"))
from scripts.utils import to_snake_case, clean_whitespace, remove_outlier

In [2]:
# Configurar pandas para exibir todas as colunas
pd.set_option('display.max_columns', None)

---

# Carregar data set

In [3]:
df_demographic = pd.read_csv('../../data/raw/DemographicData_ZCTAs.csv')
df_demographic.head()

,Unnamed: 0,Id,GeographicAreaName,TotalPopulation,SexRatio(males per 100 females),Population_Under5Years,Population_5to9Years,Population_10to14Years,Population_15to19Years,Population_20to24Years,Population_25to34Years,Population_35to44Years,Population_45to54Years,Population_55to59Years,Population_60to64Years,Population_65to74Years,Population_75to84Years,Population_85YearsAndOver,MedianAgeInYears
0,0,8600000US35004,ZCTA5 35004,12045,94.1,805,1075,898,477,578,2088,1628,1200,886,683,1017,534,176,35.5
1,1,8600000US35005,ZCTA5 35005,7344,86.1,504,453,511,499,214,788,795,968,612,561,798,485,156,44.0
2,2,8600000US35006,ZCTA5 35006,2883,108.2,96,153,303,129,156,183,367,430,296,260,280,201,29,47.2
3,3,8600000US35007,ZCTA5 35007,26332,95.0,1936,1992,1837,1762,1376,3119,3849,3907,1665,1323,2096,1106,364,37.7
4,4,8600000US35010,ZCTA5 35010,20613,90.5,1306,1465,944,1217,1128,2513,2106,2950,1512,1472,2421,1155,424,42.6


In [4]:
# Deletar a coluna 'Unnamed: 0' (apenas é um índice)
df_demographic.drop(columns=['Unnamed: 0'], inplace=True)

---

# Verificar valores duplicados

In [5]:
# Verificando a quantidade de linhas duplicadas
df_demographic.duplicated().sum()

np.int64(0)

---

# Analisar os tipos de dados

In [6]:
df_demographic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33120 entries, 0 to 33119
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Id                               33120 non-null  object 
 1   GeographicAreaName               33120 non-null  object 
 2   TotalPopulation                  33120 non-null  int64  
 3   SexRatio(males per 100 females)  32620 non-null  float64
 4   Population_Under5Years           33120 non-null  int64  
 5   Population_5to9Years             33120 non-null  int64  
 6   Population_10to14Years           33120 non-null  int64  
 7   Population_15to19Years           33120 non-null  int64  
 8   Population_20to24Years           33120 non-null  int64  
 9   Population_25to34Years           33120 non-null  int64  
 10  Population_35to44Years           33120 non-null  int64  
 11  Population_45to54Years           33120 non-null  int64  
 12  Population_55to59Y

Todas as colunas aparentam a tipagem correta, no entanto imaginando criar a pipeline, pode-se forçar a tipagem das colunas que contenha a string `Population`

In [7]:
num_cols = [col for col in df_demographic.columns if col.startswith('Population')]
df_demographic[num_cols] = df_demographic[num_cols].apply(pd.to_numeric, errors='coerce')

In [8]:
# Verificando novamente o tipo de dados
df_demographic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33120 entries, 0 to 33119
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Id                               33120 non-null  object 
 1   GeographicAreaName               33120 non-null  object 
 2   TotalPopulation                  33120 non-null  int64  
 3   SexRatio(males per 100 females)  32620 non-null  float64
 4   Population_Under5Years           33120 non-null  int64  
 5   Population_5to9Years             33120 non-null  int64  
 6   Population_10to14Years           33120 non-null  int64  
 7   Population_15to19Years           33120 non-null  int64  
 8   Population_20to24Years           33120 non-null  int64  
 9   Population_25to34Years           33120 non-null  int64  
 10  Population_35to44Years           33120 non-null  int64  
 11  Population_45to54Years           33120 non-null  int64  
 12  Population_55to59Y

---

# Verificar valores nulos e ausentes

In [9]:
# Verificando a quantidade de valores nulos
df_demographic.isnull().sum()

Id                                   0
GeographicAreaName                   0
TotalPopulation                      0
SexRatio(males per 100 females)    500
Population_Under5Years               0
Population_5to9Years                 0
Population_10to14Years               0
Population_15to19Years               0
Population_20to24Years               0
Population_25to34Years               0
Population_35to44Years               0
Population_45to54Years               0
Population_55to59Years               0
Population_60to64Years               0
Population_65to74Years               0
Population_75to84Years               0
Population_85YearsAndOver            0
MedianAgeInYears                   569
dtype: int64

In [10]:
# Verificando a quantidade de valores ausentes
df_demographic.isna().sum()

Id                                   0
GeographicAreaName                   0
TotalPopulation                      0
SexRatio(males per 100 females)    500
Population_Under5Years               0
Population_5to9Years                 0
Population_10to14Years               0
Population_15to19Years               0
Population_20to24Years               0
Population_25to34Years               0
Population_35to44Years               0
Population_45to54Years               0
Population_55to59Years               0
Population_60to64Years               0
Population_65to74Years               0
Population_75to84Years               0
Population_85YearsAndOver            0
MedianAgeInYears                   569
dtype: int64

In [11]:
# Verificando os valores ausentes de 'SexRatio(males per 100 females)'
df_demographic[df_demographic['SexRatio(males per 100 females)'].isna()]

,Id,GeographicAreaName,TotalPopulation,SexRatio(males per 100 females),Population_Under5Years,Population_5to9Years,Population_10to14Years,Population_15to19Years,Population_20to24Years,Population_25to34Years,Population_35to44Years,Population_45to54Years,Population_55to59Years,Population_60to64Years,Population_65to74Years,Population_75to84Years,Population_85YearsAndOver,MedianAgeInYears
5,8600000US35013,ZCTA5 35013,46,NaN,0,0,0,0,0,0,15,31,0,0,0,0,0,47.9
47,8600000US35082,ZCTA5 35082,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
142,8600000US35457,ZCTA5 35457,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
403,8600000US36267,ZCTA5 36267,7,NaN,0,0,0,0,0,0,0,0,7,0,0,0,0,NaN
539,8600000US36590,ZCTA5 36590,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32920,8600000US82715,ZCTA5 82715,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
32955,8600000US82936,ZCTA5 82936,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
32981,8600000US83121,ZCTA5 83121,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
33099,8600000US00950,ZCTA5 00950,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN


Como o dado aparenta não ser relavante, pode-se deletar

In [12]:
# Deletando os valores ausentes de 'SexRatio(males per 100 females)'
df_demographic.dropna(subset=['SexRatio(males per 100 females)'], inplace=True)

In [13]:
# Verificando novamente
df_demographic[df_demographic['SexRatio(males per 100 females)'].isna()]

,Id,GeographicAreaName,TotalPopulation,SexRatio(males per 100 females),Population_Under5Years,Population_5to9Years,Population_10to14Years,Population_15to19Years,Population_20to24Years,Population_25to34Years,Population_35to44Years,Population_45to54Years,Population_55to59Years,Population_60to64Years,Population_65to74Years,Population_75to84Years,Population_85YearsAndOver,MedianAgeInYears


---

# Remover outliers

In [14]:
df_demographic.describe()

,TotalPopulation,SexRatio(males per 100 females),Population_Under5Years,Population_5to9Years,Population_10to14Years,Population_15to19Years,Population_20to24Years,Population_25to34Years,Population_35to44Years,Population_45to54Years,Population_55to59Years,Population_60to64Years,Population_65to74Years,Population_75to84Years,Population_85YearsAndOver,MedianAgeInYears
count,32620.000000,32620.000000,32620.000000,32620.000000,32620.000000,32620.000000,32620.000000,32620.000000,32620.000000,32620.000000,32620.000000,32620.000000,32620.000000,32620.000000,32620.000000,32503.000000
mean,10054.024372,116.842275,610.375169,623.294758,647.640313,656.849540,681.800307,1392.667903,1268.561435,1302.908768,670.402269,622.740711,916.701533,465.486052,194.595616,42.701018
std,14775.502970,1483.160382,1000.754249,999.871869,1030.595489,1055.807952,1243.279669,2337.849699,2003.216409,1933.310096,942.862340,860.545974,1273.817940,684.121144,314.031270,9.091496
min,3.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.900000
25%,751.000000,91.300000,34.000000,37.000000,40.000000,41.000000,33.000000,71.000000,75.000000,92.000000,55.000000,55.000000,83.000000,41.000000,10.000000,37.100000
50%,2924.000000,98.400000,152.000000,166.000000,181.000000,179.000000,156.000000,311.000000,327.500000,381.000000,219.000000,210.000000,316.000000,157.000000,53.500000,42.000000
75%,13773.000000,107.900000,764.000000,799.000000,833.000000,839.000000,801.000000,1659.000000,1630.000000,1771.000000,941.000000,884.000000,1295.000000,637.000000,247.000000,47.600000
max,128294.000000,265600.000000,17564.000000,14784.000000,12289.000000,13357.000000,23027.000000,28192.000000,23166.000000,18058.000000,7392.000000,7018.000000,26465.000000,16934.000000,4633.000000,98.200000


Avalidando os extremos do `SexRatio(males per 100 females)` (0.5 - 265600), não faz muito sentido para o que representa a coluna

In [15]:
# Portanto devemos remover os outliers
df_demographic = remove_outlier(df_demographic, 'SexRatio(males per 100 females)')

In [16]:
df_demographic.describe()

,TotalPopulation,SexRatio(males per 100 females),Population_Under5Years,Population_5to9Years,Population_10to14Years,Population_15to19Years,Population_20to24Years,Population_25to34Years,Population_35to44Years,Population_45to54Years,Population_55to59Years,Population_60to64Years,Population_65to74Years,Population_75to84Years,Population_85YearsAndOver,MedianAgeInYears
count,29528.000000,29528.000000,29528.000000,29528.000000,29528.000000,29528.000000,29528.000000,29528.000000,29528.000000,29528.000000,29528.000000,29528.000000,29528.000000,29528.000000,29528.000000,29432.000000
mean,10934.688939,98.749075,666.024688,680.650163,707.521945,712.614772,735.142746,1509.055473,1378.432031,1418.001456,730.049919,678.592624,998.922209,507.354714,212.326199,42.565272
std,15200.990176,11.899875,1033.014959,1031.639280,1063.125062,1086.903722,1281.397944,2412.882123,2064.652185,1988.406034,967.841878,882.684975,1306.782731,703.087628,323.510455,8.406964
min,5.000000,66.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.600000
25%,995.000000,91.300000,48.000000,52.000000,56.000000,55.000000,44.000000,95.000000,102.000000,125.000000,74.000000,74.000000,113.000000,55.000000,15.000000,37.200000
50%,3561.500000,97.700000,191.000000,208.000000,225.000000,221.000000,188.000000,378.000000,402.500000,477.000000,268.000000,261.000000,391.000000,193.500000,67.000000,41.900000
75%,15793.000000,105.500000,881.250000,917.250000,962.000000,951.000000,905.000000,1917.000000,1884.000000,2019.000000,1067.000000,1001.000000,1462.000000,723.000000,284.000000,47.200000
max,128294.000000,132.800000,17564.000000,14784.000000,12289.000000,13357.000000,23027.000000,28192.000000,23166.000000,18058.000000,7392.000000,7018.000000,26465.000000,16934.000000,4633.000000,91.500000


Agora o intervalo (66 - 132) faz muito mais sentido para o sex ratio

# Organizar os dados

Como a coluna `GeographicAreaName` contém o valor de zipcode, é realizado a extração.

In [17]:
df_demographic['zipcode'] = df_demographic['GeographicAreaName'].str.extract(r'(\d{5})')

# Como a coluna 'GeographicAreaName' não é mais necessária, pode-se deletá-la
df_demographic.drop(columns='GeographicAreaName', inplace=True)

df_demographic.head()

,Id,TotalPopulation,SexRatio(males per 100 females),Population_Under5Years,Population_5to9Years,Population_10to14Years,Population_15to19Years,Population_20to24Years,Population_25to34Years,Population_35to44Years,Population_45to54Years,Population_55to59Years,Population_60to64Years,Population_65to74Years,Population_75to84Years,Population_85YearsAndOver,MedianAgeInYears,zipcode
0,8600000US35004,12045,94.1,805,1075,898,477,578,2088,1628,1200,886,683,1017,534,176,35.5,35004
1,8600000US35005,7344,86.1,504,453,511,499,214,788,795,968,612,561,798,485,156,44.0,35005
2,8600000US35006,2883,108.2,96,153,303,129,156,183,367,430,296,260,280,201,29,47.2,35006
3,8600000US35007,26332,95.0,1936,1992,1837,1762,1376,3119,3849,3907,1665,1323,2096,1106,364,37.7,35007
4,8600000US35010,20613,90.5,1306,1465,944,1217,1128,2513,2106,2950,1512,1472,2421,1155,424,42.6,35010


In [18]:
# Com o objetivo de resduzir o número de colunas e facilitar a análise, pode-se transformar as colunas que possuem o mesmo prefixo (`Population`) em uma única coluna
df_demographic = pd.melt(
    df_demographic,
    id_vars=['Id', 'zipcode', 'TotalPopulation', 'SexRatio(males per 100 females)', 'MedianAgeInYears'],
    value_vars=num_cols,
    var_name='AgeGroup',
    value_name='group_population'
)

In [19]:
# Verificando o resultado
df_demographic.head()

,Id,zipcode,TotalPopulation,SexRatio(males per 100 females),MedianAgeInYears,AgeGroup,group_population
0,8600000US35004,35004,12045,94.1,35.5,Population_Under5Years,805
1,8600000US35005,35005,7344,86.1,44.0,Population_Under5Years,504
2,8600000US35006,35006,2883,108.2,47.2,Population_Under5Years,96
3,8600000US35007,35007,26332,95.0,37.7,Population_Under5Years,1936
4,8600000US35010,35010,20613,90.5,42.6,Population_Under5Years,1306


In [20]:
# Melhorando a legibilidade dos dados
df_demographic['AgeGroup'] = (
    df_demographic['AgeGroup']
    .str.replace('Population_', '', regex=False)
    .str.replace('to', ' - ', regex=False)
    .str.replace('Under', '< ', regex=False)
    .str.replace('AndOver', ' >=', regex=False)
    .str.replace('Years', ' Years', regex=False)
)

# Transformações Gerais

In [21]:
# Transformando as colunas em snake_case
to_snake_case(df_demographic)

# Limpando os espaços em branco
clean_whitespace(df_demographic)

In [22]:
df_demographic.head()

,id,zipcode,total_population,sex_ratiomalesper100females,median_age_in_years,age_group,group_population
0,8600000US35004,35004,12045,94.1,35.5,< 5 Years,805
1,8600000US35005,35005,7344,86.1,44.0,< 5 Years,504
2,8600000US35006,35006,2883,108.2,47.2,< 5 Years,96
3,8600000US35007,35007,26332,95.0,37.7,< 5 Years,1936
4,8600000US35010,35010,20613,90.5,42.6,< 5 Years,1306


---

# Validação Final

In [23]:
df_demographic.describe(include='all')

,id,zipcode,total_population,sex_ratiomalesper100females,median_age_in_years,age_group,group_population
count,383864,383864,383864.000000,383864.000000,382616.000000,383864,383864.000000
unique,29528,29528,NaN,NaN,NaN,13,NaN
top,8600000US00987,00987,NaN,NaN,NaN,< 5 Years,NaN
freq,13,13,NaN,NaN,NaN,29528,NaN
mean,NaN,NaN,10934.688939,98.749075,42.565272,NaN,841.129918
std,NaN,NaN,15200.752574,11.899689,8.406832,NaN,1411.092467
min,NaN,NaN,5.000000,66.400000,4.600000,NaN,0.000000
25%,NaN,NaN,995.000000,91.300000,37.200000,NaN,61.000000
50%,NaN,NaN,3561.500000,97.700000,41.900000,NaN,245.000000
75%,NaN,NaN,15793.000000,105.500000,47.200000,NaN,1020.000000


---

# Exportar data set limpo

In [24]:
df_demographic.to_csv('../../data/processed/demographic_data_clean.csv', index=False)